## Real Time Testing

In [9]:
from collections import deque
from concurrent.futures import ThreadPoolExecutor
from time import time
from cvzone import FPS

import cv2
import numpy as np
from itertools import chain
import traceback
from time import time
import os

from cvzone.HandTrackingModule import HandDetector
from cvzone.FaceDetectionModule import FaceDetector
from cvzone.PoseModule import PoseDetector

In [10]:
## 2. Feature Extraction (Hand+Face+Pose Detection)
# Flatten a 2d np array into 1d array
def flatten2dList(arr, dataType=int):
    return np.fromiter(chain.from_iterable(arr), dataType)

# Get the largest absolute value in an np array
def getAbsLargestVal(arr):
    return np.max(np.abs(arr))

# Offset and normalize the landmark list
# Returns a 1d numpy array
def preprocess_landmarks(landmark_list):    
    landmark_list = np.array(landmark_list, dtype=float)
    origin = landmark_list[0]
    
    # Offset every point with respect to the first point
    # Convert to 1D-array
    new_landmark_list = (landmark_list - origin).ravel()
    
    # Get highest absolute value
    largest_value = getAbsLargestVal(new_landmark_list)
    
    # Normalization
    if largest_value != 0:
        return new_landmark_list / largest_value
    return new_landmark_list

# Offset and normalize a BBOX list (BBOX = Bounding Box, used in face and hand detection)
# Returns a 1d numpy array
def preprocess_bbox(bbox, frameSize):
    bbox = np.array(bbox, dtype=float)
    # Convert 3rd and 4th element into coordinates instead of width/height
    bbox[2] = bbox[0] + bbox[2]
    bbox[3] = bbox[1] + bbox[3]

    # Normalize against frame size
    bbox[0] /= frameSize[0]
    bbox[1] /= frameSize[1]
    bbox[2] /= frameSize[0]
    bbox[3] /= frameSize[1]

    return bbox

# Normalize a center vertex (a list of 2 elements)
# Returns a 1d numpy array
def preprocess_center(center, frameSize):
    center = np.array(center, dtype=float)
    center[0] /= frameSize[0]
    center[1] /= frameSize[1]
    return center

# Preprocess (Offset and normalize) the body's landmark list, bbox and center
def preprocess_body_part(bodyPart, frameSize):
    bodyPart['lmList'] = preprocess_landmarks(bodyPart['lmList'])
    bodyPart['bbox'] = preprocess_bbox(bodyPart['bbox'], frameSize)
    bodyPart['center'] = preprocess_center(bodyPart['center'], frameSize)
    return bodyPart

# Function to generate empty/placeholder data for a hand 
# Used when a hand is not detected in frame
def generate_empty_hand(type):
    return {
        'lmList': np.zeros(21 * 3, dtype=int), 
        'bbox': np.zeros(4, dtype=float), 
        'center': np.zeros(2, dtype=float), 
        'type': type
    }

# Select the best matching face, aka the one with the best score (clarity)
# and closest to the center of the screen
# Since the Neural network will be design to only accept one face
def select_best_matching_face(faces, frameSize):
    if not faces or len(faces) == 0:
        return False
    elif len(faces) == 1:
        return faces[0]
    
    def difference(a, b):
        return ((a[0] - b[0])**2) + ((a[1] - b[1])**2)
    
    frameCenter = (frameSize[0] / 2, frameSize[1] / 2)

    best_score = faces[0]
    best_center = faces[0]
    center_diff = difference(faces[0]['center'], frameCenter)

    for each in faces:
        if difference(each['center'], frameCenter) < center_diff:
            best_center = each
        if each['score'][0] > best_score['score'][0]:
            best_score = each
    
    if best_center['score'][0] > 0.5:
        return best_center
    return best_score

# Flatten everything
def flattenDetectionResult(obj):
    # return np.fromiter(chain.from_iterable([obj['lmList'], obj['bbox'], obj['center']]), float)
    return np.concatenate([obj['lmList'], obj['bbox'], obj['center']])

In [11]:
# Detects hands, face & pose, 
# convert them into normalized landmark/keypoint coordinates in a 1D-array, 
# and also returns the frame with the landmark connections drawn onto it

# Improved/Parallelised version
def featureExtractionV3(handDetector, faceDetector, poseDetector, frame, draw=True):
    def detectHands(handDetector, frame, frameSize, draw):
        results = None
        # Hand Detection
        if (draw):
            results, frame = handDetector.findHands(frame, draw=draw)
        else:
            results = handDetector.findHands(frame, draw=draw)

        if not results:
            results = [generate_empty_hand('Left'), generate_empty_hand('Right')]
        elif len(results) == 1:
            if (results[0]['type'] == 'Left'):
                results[0] = preprocess_body_part(results[0], frameSize)
                results.append(generate_empty_hand('Right'))
            else:
                results[0] = preprocess_body_part(results[0], frameSize)
                results.insert(0, generate_empty_hand('Left'))                         
        else:
            results[0] = preprocess_body_part(results[0], frameSize)
            results[1] = preprocess_body_part(results[1], frameSize)
        return results

    # Pose Detection
    # **We only use the first 23 out of the total 33 landmark points 
    #   as those represent the lower half body and are irrelevant to sign language interpretation
    def detectPose(poseDetector, frame, draw):
        frame = poseDetector.findPose(frame, draw=draw)
        results, _ = poseDetector.findPosition(frame, bboxWithHands=False)
        if results:
            results = preprocess_landmarks(results[:23])
        else:
            results = np.zeros(23, dtype=int)
        return results
    
    # Face Detection
    def detectFace(faceDetector, frame, frameSize, draw):
        frame, results = faceDetector.findFaces(frame, draw=draw)
        if results:
            results = select_best_matching_face(results, frameSize)
            results['bbox'] = preprocess_bbox(results['bbox'], frameSize)
            results['center'] = preprocess_center(results['center'], frameSize)
        else:
            results = {
                'bbox': np.zeros(4, dtype=float), 
                'center': np.zeros(2, dtype=float)
            }
        return results

    frameSize = (frame.shape[1], frame.shape[0])
    with ThreadPoolExecutor() as executor:
        t1 = executor.submit(detectHands, handDetector, frame, frameSize, draw)
        t2 = executor.submit(detectPose, poseDetector, frame, draw)
        t3 = executor.submit(detectFace, faceDetector, frame, frameSize, draw)
        
        # Convert results into 1D-array
        detectionResults = flatten2dList([
            flattenDetectionResult(t1.result()[0]), 
            flattenDetectionResult(t1.result()[1]), 
            t2.result(), 
            t3.result()['bbox'],
            t3.result()['center'],
            t3.result()['center'] - t1.result()[0]['center'],
            t3.result()['center'] - t1.result()[1]['center']
        ], dataType=float)

        return detectionResults, frame

In [12]:
# Detects hands, face & pose, 
# convert them into normalized landmark/keypoint coordinates in a 1D-array, 
# and also returns the frame with the landmark connections drawn onto it

# Improved/Parallelised version
def staticFeatureExtraction(handDetector, frame, draw=True):
    def detectHands(handDetector, frame, frameSize, draw):
        results = None
        # Hand Detection
        if (draw):
            results, frame = handDetector.findHands(frame, draw=draw)
        else:
            results = handDetector.findHands(frame, draw=draw)

        if not results:
            results = [generate_empty_hand('Left'), generate_empty_hand('Right')]
        elif len(results) == 1:
            if (results[0]['type'] == 'Left'):
                results[0] = preprocess_body_part(results[0], frameSize)
                results.append(generate_empty_hand('Right'))
            else:
                results[0] = preprocess_body_part(results[0], frameSize)
                results.insert(0, generate_empty_hand('Left'))                         
        else:
            results[0] = preprocess_body_part(results[0], frameSize)
            results[1] = preprocess_body_part(results[1], frameSize)
        return results

    frameSize = (frame.shape[1], frame.shape[0])
    with ThreadPoolExecutor() as executor:
        t1 = executor.submit(detectHands, handDetector, frame, frameSize, draw)
        # t2 = executor.submit(detectPose, poseDetector, frame, draw)
        # t3 = executor.submit(detectFace, faceDetector, frame, frameSize, draw)
        
        # Convert results into 1D-array
        detectionResults = flatten2dList([
            flattenDetectionResult(t1.result()[0]), 
            flattenDetectionResult(t1.result()[1]), 
            # t2.result(), 
            # t3.result()['bbox'],
            # t3.result()['center'],
            # t3.result()['center'] - t1.result()[0]['center'],
            # t3.result()['center'] - t1.result()[1]['center']
        ], dataType=float)

        return detectionResults, frame

In [13]:
import tensorflow as tf
import keras

# Load the model from the H5 file
model = tf.keras.models.load_model('../static_recognition/models/static_model.h5')
# model = tf.keras.models.load_model('../static_recognition/models/static_model_hand_only.h5')
from files_io import readActionLabels

static_labels = readActionLabels()
static_labels

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 'airplane']

In [14]:
# cam = cv2.VideoCapture(0, cv2.CAP_DSHOW)
# cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
# cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# # Detectors
# handDetector = HandDetector(detectionCon=0.5, maxHands=2)

# fpsReader = FPS()

# timeStats = []

# try:
#     predictionHistory = deque()
#     detectionThreshold = 0.999    
#     predictionCooldown = 1.0
#     # the first time append should eliminate predictionCooldown
#     lastAppendTime = time() + predictionCooldown

#     while True:
#         startTime = time()

#         # Read from camera
#         success, frame = cam.read()
#         if not success:
#             raise Exception("No Frames Read")
#         frame = cv2.flip(frame, 1)

#         # Pose Detection
#         detectionResults, frame = staticFeatureExtraction(
#             handDetector, frame)
        
#         detectionResults, frame = staticFeatureExtraction(
#             handDetector, frame)

#         detectionResults = np.expand_dims(
#             detectionResults, axis=0) # reshape detection result

#         predictionResults = model.predict(
#             x=detectionResults,
#             verbose=0,
#             use_multiprocessing=True,
#             workers=4
#         )[0]
        
#         # Get predicted character and accuracy
#         predCharacter = static_labels[np.argmax(predictionResults)]
#         predAccuracy = predictionResults[np.argmax(predictionResults)]
        
#         cv2.putText(frame, (predCharacter + ":"+ str(predAccuracy)), (15, 70),
#                 cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
        
#         if predAccuracy >= detectionThreshold:            
#             # If predictionHistory is not empty
#             # If predCharacter is the same as the last appended character
#             # Check if 0.5 seconds have passed since the last append
#             if predictionHistory and predCharacter == predictionHistory[-1] and time() <= lastAppendTime + predictionCooldown:
#                 # Do nothing, don't append
#                 pass
#             else:
#                 predictionHistory.append(predCharacter)
#                 # Reset the timestamp when a new character is detected
#                 lastAppendTime = time()

#             if len(predictionHistory) > 5:
#                     predictionHistory.popleft()  
                           
                        
#         cv2.putText(frame, str(predictionHistory), (15, 120),
#                     cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
        

#         fps, frame = fpsReader.update(frame, pos=(
#             950, 80), color=(0, 255, 0), scale=5, thickness=5)

#         # Show resulting frame
#         # cv2.putText(frame, f'Training #{training + 1} for \'{action}\'', (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
#         cv2.imshow("Sign Language Recognition Prototype", frame)

#         timeStats.append(time() - startTime)

#         keyPressed = cv2.waitKey(1)
#         # Stop Program when pressed 'Esc'
#         if keyPressed == 27:
#             raise Exception("Finished")
#         elif keyPressed == ord('c'):
#             predictionHistory.clear()
#             lastAppendTime = time() + predictionCooldown


# except Exception as e:
#     cam.release()
#     cv2.destroyAllWindows()
#     print(e)

# cam.release()
# cv2.destroyAllWindows()

In [15]:
cam = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# Detectors
handDetector = HandDetector(detectionCon=0.5, maxHands=2)
faceDetector = FaceDetector(minDetectionCon=0.5)
poseDetector = PoseDetector(detectionCon=0.5)

fpsReader = FPS()

timeStats = []

try:
    keypointsHistory = deque()
    predictionHistory = deque()
    detectionThreshold = 0.999

    lastPredictionTime = time()
    lastAppendTime = time()
    predictionCooldown = 0.7

    while True:
        startTime = time()

        # Read from camera
        success, frame = cam.read()
        if not success:
            raise Exception("No Frames Read")
        frame = cv2.flip(frame, 1)

        # Pose Detection
        detectionResults, frame = featureExtractionV3(
            handDetector, faceDetector, poseDetector, frame)

        detectionResults = np.expand_dims(
            detectionResults, axis=0)  # Reshape to (1, 138, 3)

        # Semantic Prediction
        keypointsHistory.append(detectionResults)
        
        if time() > lastPredictionTime + predictionCooldown:
            predictionResults = model.predict(
                x=detectionResults,
                verbose=0,
                use_multiprocessing=True,
                workers=4
            )[0]

            predCharacter = static_labels[np.argmax(predictionResults)]
            predAccuracy = predictionResults[np.argmax(predictionResults)]
            
            cv2.putText(frame, (predCharacter + ":"+ str(predAccuracy)), (15, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
            
            if predAccuracy >= detectionThreshold:                
                # If predictionHistory is not empty
                # If predCharacter is the same as the last appended character
                if predictionHistory and predCharacter == predictionHistory[-1]:
                    # Check if 0.5 seconds have passed since the last append
                    if time() > lastAppendTime + predictionCooldown:
                        predictionHistory.append(predCharacter)
                        # Reset the timestamp when a new character is detected
                        lastAppendTime = time()
                else:
                    # do not need to check append time, directly append
                    predictionHistory.append(predCharacter)
                    # Reset the timestamp when a new character is detected
                    lastAppendTime = time()
                    
                if len(predictionHistory) > 5:
                        predictionHistory.popleft()      
                        
        cv2.putText(frame, str(predictionHistory), (15, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
        

        fps, frame = fpsReader.update(frame, pos=(
            950, 80), color=(0, 255, 0), scale=5, thickness=5)

        # Show resulting frame
        # cv2.putText(frame, f'Training #{training + 1} for \'{action}\'', (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
        cv2.imshow("Sign Language Recognition Prototype", frame)

        timeStats.append(time() - startTime)

        keyPressed = cv2.waitKey(1)
        # Stop Program when pressed 'Esc'
        if keyPressed == 27:
            raise Exception("Finished")
        elif keyPressed == ord('c'):
            predictionHistory.clear()


except Exception as e:
    cam.release()
    cv2.destroyAllWindows()
    print(e)

cam.release()
cv2.destroyAllWindows()

Finished


In [16]:
timeStats[10:]

[0.14885950088500977,
 0.11977934837341309,
 0.13950753211975098,
 0.11876416206359863,
 0.13624930381774902,
 0.11857056617736816,
 0.1325688362121582,
 0.13489270210266113,
 0.14676856994628906,
 0.1379561424255371,
 0.13553166389465332,
 0.14408588409423828,
 0.14115071296691895,
 0.11650514602661133,
 0.13970232009887695,
 0.14253664016723633,
 0.14525580406188965,
 0.1394493579864502,
 0.12318301200866699,
 0.152055025100708,
 0.15252280235290527,
 0.1175835132598877,
 0.13906121253967285,
 0.1141972541809082,
 0.1026010513305664,
 0.1065526008605957,
 0.10360383987426758,
 0.10651183128356934,
 0.10342264175415039,
 0.10581350326538086,
 0.1122744083404541,
 0.1047523021697998,
 0.10457706451416016,
 0.10786867141723633,
 0.10352778434753418,
 0.10556221008300781,
 0.10486149787902832,
 0.10865020751953125,
 0.1143639087677002,
 0.10972905158996582,
 0.10373401641845703,
 0.10738301277160645,
 0.10907769203186035,
 0.1051175594329834,
 0.10578441619873047,
 0.10781669616699219,
 